# Deep Learning: CNN: Cancer Detection
**Thomas Bohn**   --   **2025-09-14**

{{xxxxx}}  

--  [Main Report](xxxx)  --  [Github Repo](xxxx)  --  [Presentation Slides](xxx)  --  [Presentation Video](xxx) --  

# 1.&nbsp;Introduction

**Problem Statement**

{{xxxxx}}

**Why is it Important?**

{{xxxxx}}

**Limitations of Existing Solutions**

{{xxxxx}}

**Contribution**

{{xxxxx}}

**DataSet**

{{xxxxx}}


## Python Libraries

The following python libraries are used in this notebook.

In [ ]:
# File system manangement
import time, datetime, psutil, os
import shutil
import zipfile
from google.colab import userdata
from google.colab import files

# Data manipulation
import numpy as np
import pandas as pd
import math

# Install text storage and manipulation
import re
import json
import pickle
import textwrap

#Install Image processing
from PIL import Image

##################################

# Plotting and visualization
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
sns.set_theme()

# Train-test split and cross validation
from sklearn.model_selection import train_test_split, ParameterGrid

# Model evaluation
from sklearn import metrics
from sklearn.metrics import accuracy_score

#################################

# Mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
#################################

# Import Tensor Flow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
import numpy as np

#################################

## Connect to TPUs

In [ ]:
# TPU (Tensor Processing Unit) Setup for Accelerated Training
# This code attempts to connect to Google's TPU infrastructure for faster model training
# TPUs are specialized hardware designed specifically for machine learning workloads

try:
    # Try to detect and connect to a TPU cluster
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())  # Print the TPU master address
    
    # Connect to the TPU cluster
    tf.config.experimental_connect_to_cluster(tpu)
    
    # Initialize the TPU system for use
    tf.tpu.experimental.initialize_tpu_system(tpu)
    
    # Create a TPU distribution strategy for multi-core TPU usage
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
except:
    # Fallback: If TPU is not available, use the default strategy (CPU/GPU)
    # This ensures the code works in environments without TPU access
    strategy = tf.distribute.get_strategy()

# Print the number of replicas (cores) available for parallel processing
print('Number of replicas:', strategy.num_replicas_in_sync)

# Set up automatic tuning for data pipeline performance optimization
# AUTOTUNE allows TensorFlow to automatically determine the optimal number of parallel calls
AUTOTUNE = tf.data.experimental.AUTOTUNE
    
# Print TensorFlow version for reference
print(tf.__version__)

## Global Variables

The following are global variables referenced in this notebook.

In [2]:
# Recording the starting time, complemented with a stopping time check in the end to compute process runtime
start = time.time()

# Class representing the OS process and having memory_info() method to compute process memory usage
process = psutil.Process(os.getpid())

In [3]:
# Global Debug flag used to turn on and off more chatty blocks of code
gDEBUG = False
if gDEBUG: print('Debug is set to:', gDEBUG)
# Global Level of Detail of table stats and details
gLOD = 2
print('Level of Detail for functions is set to:', gLOD)

Level of Detail for functions is set to: 2


# 2.&nbsp;Data Source

In this section, the code loads the dataset from Google Drive.

{{xxxxx}}

## Import the Data (Kaggle)

We want to keep our photo dataset and our Monet dataset separate. First, load in the filenames of the TFRecords.

In [ ]:
# Dataset Path Configuration for Kaggle Environment
# Get the Google Cloud Storage (GCS) path for Kaggle datasets
# This allows access to the competition datasets stored in Kaggle's cloud storage
GCS_PATH = KaggleDatasets().get_gcs_path()

# Load Monet Painting Dataset
# Search for all TFRecord files containing Monet paintings in the monet_tfrec directory
# TFRecord is TensorFlow's efficient binary format for storing large datasets
MONET_FILENAMES = tf.io.gfile.glob(str(GCS_PATH + '/monet_tfrec/*.tfrec'))
print('Monet TFRecord Files:', len(MONET_FILENAMES))

# Load Photo Dataset  
# Search for all TFRecord files containing regular photos in the photo_tfrec directory
# These photos will be transformed into Monet-style paintings by the GAN
PHOTO_FILENAMES = tf.io.gfile.glob(str(GCS_PATH + '/photo_tfrec/*.tfrec'))
print('Photo TFRecord Files:', len(PHOTO_FILENAMES))

## Import the Data (Colab)


In [ ]:

# Define the source of the zipped data files
target_file = 'gan-getting-started.zip'
source_path_root =  '/content/drive/MyDrive/[1.4] MsDS Class Files/-- DTSA 5511 Deep Learning/data'
destination_path_root = '/content'

# Copy the files to the runtime
shutil.copy(source_path_root+'/'+target_file, destination_path_root+'/')

# Display the files in the destination directory
print(os.listdir(destination_path_root+'/'))

# Unzip the files (this is slow)
zip_file_path = destination_path_root+'/'+target_file

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all the contents into the specified folder
        zip_ref.extractall(destination_path_root)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Dataset Path Configuration for Google Colab Environment
# Set up local file paths for the extracted dataset files
# This replicates the Kaggle GCS_PATH functionality for local Colab environment

# Define the base path where the dataset was extracted
COLAB_DATA_PATH = '/content/gan-getting-started'

# Load Monet Painting Dataset (Colab Version)
# Search for all TFRecord files containing Monet paintings in the monet_tfrec directory
# Using os.path.join for cross-platform compatibility
MONET_FILENAMES = tf.io.gfile.glob(os.path.join(COLAB_DATA_PATH, 'monet_tfrec', '*.tfrec'))
print('Monet TFRecord Files:', len(MONET_FILENAMES))

# Load Photo Dataset (Colab Version)
# Search for all TFRecord files containing regular photos in the photo_tfrec directory
# These photos will be transformed into Monet-style paintings by the GAN
PHOTO_FILENAMES = tf.io.gfile.glob(os.path.join(COLAB_DATA_PATH, 'photo_tfrec', '*.tfrec'))
print('Photo TFRecord Files:', len(PHOTO_FILENAMES))

# Verify the datasets are loaded correctly
if len(MONET_FILENAMES) > 0:
    print(f"✅ Successfully loaded {len(MONET_FILENAMES)} Monet painting files")
    print(f"First Monet file: {MONET_FILENAMES[0]}")
else:
    print("❌ No Monet files found. Check the dataset extraction path.")

if len(PHOTO_FILENAMES) > 0:
    print(f"✅ Successfully loaded {len(PHOTO_FILENAMES)} photo files")
    print(f"First photo file: {PHOTO_FILENAMES[0]}")
else:
    print("❌ No photo files found. Check the dataset extraction path.")

## Load the Datasets

All the images for the competition are already sized to 256x256. As these images are RGB images, set the channel to 3. Additionally, we need to scale the images to a [-1, 1] scale. Because we are building a generative model, we don't need the labels or the image id so we'll only return the image from the TFRecord. Finally, we define the function to extract the image from the files.

In [ ]:
# Image Processing Configuration
# Define the target image size for the GAN model
IMAGE_SIZE = [256, 256]

def decode_image(image):
    # Decode the JPEG image data into a tensor with 3 color channels (RGB)
    image = tf.image.decode_jpeg(image, channels=3)
    # Normalize pixel values from [0, 255] to [-1, 1] range
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    # Reshape to the target image size with 3 color channels
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

def read_tfrecord(example):
    # Define the expected structure of each TFRecord entry
    # This matches the format used in the Kaggle competition dataset
    tfrecord_format = {
        "image_name": tf.io.FixedLenFeature([], tf.string),  # Filename as string
        "image": tf.io.FixedLenFeature([], tf.string),       # Image data as bytes
        "target": tf.io.FixedLenFeature([], tf.string)       # Target label as string
    }
    # Parse the TFRecord example according to the defined format
    example = tf.io.parse_single_example(example, tfrecord_format)
    # Extract and decode the image data
    image = decode_image(example['image'])
    return image

def load_dataset(filenames, labeled=True, ordered=False):
    # Create a TFRecord dataset from the provided filenames
    dataset = tf.data.TFRecordDataset(filenames)
    # Apply the read_tfrecord function to each example in parallel
    # AUTOTUNE allows TensorFlow to automatically determine optimal parallelism
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTOTUNE)
    return dataset

In [ ]:
# Load in datasets.
monet_ds = load_dataset(MONET_FILENAMES, labeled=True).batch(1)
photo_ds = load_dataset(PHOTO_FILENAMES, labeled=True).batch(1)


## Preview Datasets

In [ ]:
# Get a sample image
example_monet = next(iter(monet_ds))
example_photo = next(iter(photo_ds))

# Visualize a photo example and a Monet example.
plt.subplot(121)
plt.title('Photo')
plt.imshow(example_photo[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Monet')
plt.imshow(example_monet[0] * 0.5 + 0.5)

# 3.&nbsp;Exploratory Data Analysis (EDA)

The EDA phase focuses on understanding the dataset, including data distribution and label counts. Various functions are used to inspect the structure of the dataset, visualize the label distribution, and assess the text length and word count of the documentation. The data is found to be somewhat imbalanced across categories.

## EDA Functions

In [ ]:
{{xxxxx}}

## EDA Analysis: Overview

In [ ]:
{{xxxxx}}

## EDA Analysis: Text Distribution

In [ ]:
{{xxxxx}}

## EDA Results

ADD HERE

# 4.&nbsp;Train-Validation-Test Split

Split the dataset into training, validation, and test sets. Use tratified splitting to ensure that the class distribution remains consistent across these sets. The distribution of records across the labels is visualized to ensure a balanced split.

## Test Split Functions

In [ ]:
{{xxxxx}}

## Test Split Analysis

In [ ]:
{{xxxxx}}

# 5.&nbsp;Data Cleansing & Text Normalization

{{xxxxx}}

## Core Normalization Functions

In [ ]:
{{xxxxx}}

## Apply Text Normalization

In [ ]:
{{xxxxx}}

# 6.&nbsp;Feature Engineering with TF-IDF

The TfidfVectorizer from scikit-learn is used to convert the text documents into numerical features. The vectorizer transforms the collection of documents into a matrix of token counts, which is then normalized using the Term Frequency-Inverse Document Frequency (TF-IDF) transformation. This matrix representation of the text data serves as input to the machine learning models.

## TF_IDF Functions

In [ ]:
{{xxxxx}}

## Vectorization

In [ ]:
{{xxxxx}}

# 7.&nbsp; Baseline Models: Supervised

## Model Functions

In [ ]:
{{xxxxx}}

## Build, Train, and Evaluate the Model

In [ ]:
{{xxxxx}}

# 8.&nbsp; Hyperparameter Tuning

## Tuning Functions

In [ ]:
{{xxxxx}}

## Execute Hyperparameter Tuning

In [ ]:
{{xxxxx}}

# 9.&nbsp;Final Prediction and Evaluation

## Evaluation Functions

In [ ]:
{{xxxxx}}

## Train the Final Model

In [ ]:
{{xxxxx}}

## Evaluate the Model

In [ ]:
{{xxxxx}}

## Explore Errors

In [ ]:
{{xxxxx}}

# 10.&nbsp;Scale the Auto-Classifier

## Auto-Classifier Functions

In [ ]:
{{xxxxx}}

## Rerun Process for L1

In [ ]:
{{xxxxx}}

## Rerun Process for L2

In [ ]:
{{xxxxx}}

# 11.&nbsp; Conclusions

{{xxxxx}}

## Results Summary

### Model Result Summary


**Baseline Results**

{{xxxxx}}

**Hyperparameter Tuning Results**

{{xxxxx}}

**Best Model Results**

{{xxxxx}}

**Best Model Performance**

{{xxxxx}}

## Model Comparison

### Model Comparisons and Findings

{{xxxxx}}

#### Baseline Results

{{xxxxx}}

#### Hyperparameter Tuning

{{xxxxx}}

#### Best Model Results

{{xxxxx}}

#### Performance Breakdown (Best Model)

{{xxxxx}}

#### Conclusion

{{xxxxx}}

## Concluding Observations

## Patterns and Conclusions Across the Models

{{xxxxx}}

# 12.&nbsp; References

**Kaggle Competition**

- [1] Amy Jang, Ana Sofia Uzsoy, and Phil Culliton. I’m Something of a Painter Myself. https://kaggle.com/competitions/gan-getting-started, 2020. Kaggle.

**Documentation and References**

- [2] Kaggle. Monet CycleGAN Tutorial. Amy Jang. 2020. https://www.kaggle.com/code/amyjang/monet-cyclegan-tutorial
- [3] Kaggle. Week 5 GAN Mini-Project - Final Execution. Ryan Lynch. 2025. https://www.kaggle.com/code/lynchrl/week-5-gan-mini-project-final-execution

**Library Documentation**

- [4] https://keras.io/examples/generative/cyclegan/
- [5] https://www.tensorflow.org/api_docs/python/tf/data/Dataset
- [6] https://www.tensorflow.org/guide/autodiff
